In [2]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

import os
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import layers, models, Input, Model
from tensorflow.keras.metrics import AUC
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Lambda
import tensorflow.keras.backend as K

In [5]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Rescaling and normalization

In [3]:
# Rescaling and normalization
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/Raw/train',
    target_size=(295, 295),
    batch_size=32,
    class_mode='binary',
    classes=['normal', 'metastasis'],
    color_mode='grayscale'
)

val_generator = val_datagen.flow_from_directory(
    '/content/drive/My Drive/Raw/val',
    target_size=(295, 295),
    batch_size=21,
    class_mode='binary',
    classes=['normal', 'metastasis'],
    color_mode='grayscale'
)


Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [ ]:

model_checkpoint = ModelCheckpoint(
    'modelo_mejor.keras', save_best_only=True, monitor='val_auc', mode='max'
)
early_stopping = EarlyStopping(monitor='val_auc', patience=5, restore_best_weights=True)


# Model 1

In [4]:
model = models.Sequential([
    layers.Input(shape=(295, 295, 1)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(295, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', AUC(name='auc')])

In [ ]:
model.summary()

history = model.fit(
    train_generator,
    epochs=10,
    callbacks = [model_checkpoint, early_stopping ],
    validation_data=val_generator,
    validation_steps=8)

model.save('model_1.h5')


In [ ]:

# Loss Plot
plt.figure(figsize=(5, 4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Accuracy Plot
plt.figure(figsize=(5, 4))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# Assuming 'model' and 'val_generator' are already defined and loaded

# Predict the entire validation set
predictions = model.predict(val_generator, steps=len(val_generator))
y_pred = (predictions > 0.9).astype(int)  # Convert probabilities to binary labels
y_true = val_generator.classes  # True labels

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

# Classification Report
cr = classification_report(y_true, y_pred)
print("Classification Report:")
print(cr)

# Calculate AUC
auc = roc_auc_score(y_true, predictions)
print("AUC: {:.2f}".format(auc))

# Model 2

In [ ]:
model = models.Sequential([
    layers.Input(shape=(295, 295, 1)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(295, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', AUC(name='auc')])

In [ ]:
model.summary()

history = model.fit(
    train_generator,
    epochs=10,
    callbacks = [model_checkpoint, early_stopping ],
    validation_data=val_generator,
    validation_steps=8)

model.save('model_2.h5')


# Model 3

In [ ]:
model = models.Sequential([
    layers.Input(shape=(295, 295, 1)),
    layers.Conv2D(32, (3, 3), use_bias=False),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), use_bias=False),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), use_bias=False),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(295, use_bias=False),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', AUC(name='auc')])


In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    callbacks = [model_checkpoint, early_stopping ],
    validation_data=val_generator,
    validation_steps=8)

model.save('model_3.h5')

# Model 4

In [ ]:
# Regularizador L2
l2_regularizer = regularizers.l2(0.001)

model = models.Sequential([
    layers.Input(shape=(295, 295, 1)),
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2_regularizer),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2_regularizer),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2_regularizer),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(512, activation='relu', kernel_regularizer=l2_regularizer),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid', kernel_regularizer=l2_regularizer)
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', AUC(name="auc")])


In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    callbacks = [model_checkpoint, early_stopping ],
    validation_data=val_generator,
    validation_steps=8)

model.save('model_4.h5')

# Model 5

In [ ]:
model = models.Sequential([
    layers.Input(shape=(295, 295, 1)),
    layers.Conv2D(32, (3, 3), use_bias=False),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), use_bias=False),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), use_bias=False),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(295, use_bias=False),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', AUC(name='auc')])


In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    callbacks = [model_checkpoint, early_stopping ],
    validation_data=val_generator,
    validation_steps=8)

model.save('model_5.h5')

# Transfer Learning

## With Weights

In [ ]:
# Duplicate input channels
def preprocess_input(x):
    return K.concatenate([x, x, x], axis=-1)

# Create base model without pre-trained weights
input_tensor = Input(shape=(295, 295, 1))
x = Lambda(preprocess_input)(input_tensor)
base_model = DenseNet121(include_top=False, weights=None, input_tensor=x)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add new layers on top of the model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(295, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_tensor, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    callbacks = [model_checkpoint, early_stopping ],
    validation_data=val_generator,
    validation_steps=8)

model.save('model_densenet_weights.h5')

## Withouth Weights

In [ ]:
base_model = DenseNet121(include_top=False, weights=None, input_tensor=x)

for layer in base_model.layers:
    layer.trainable = False


x = GlobalAveragePooling2D()(base_model.output)
x = Dense(295, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_tensor, outputs=predictions)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:
model.save('model_densenet_weights.h5')

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    callbacks = [model_checkpoint, early_stopping ],
    validation_data=val_generator,
    validation_steps=8)

model.save('model_densenet_no_weights.h5')